In [1]:
import pandas as pd
import numpy as np
import os
from utils import *
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
results = list(filter(lambda x: x != '.ipynb_checkpoints', os.listdir('results/')))

In [3]:
score = []
thres = []
model_name = []
for res in results:
    tmp = np.load('results/' + res)
    score.append(tmp[0])
    thres.append(tmp[1])
    model_name.append(res)
    
res_df = pd.DataFrame(np.array([score, thres, model_name]).T, 
                      columns=['score', 'thres', 'model_name'])

res_df['score'] = res_df['score'].astype(float)
res_df['thres'] = res_df['thres'].astype(float)

res_df = res_df.sort_values('score', ascending=False).reset_index(drop=True)
res_df = res_df[res_df['score'] > 0]

res_df['model'] = res_df['model_name'].apply(lambda x: x[:-5])
res_df = res_df.groupby('model', as_index=False).mean().sort_values('score', ascending=False).reset_index(drop=True)
res_df = res_df[:4]
res_df

,model,score,thres
0,Linknet_inceptionresnetv2_fold_,0.837810,0.433333
1,Unet_inceptionv3_fold_,0.836712,0.422500
2,Linknet_inceptionv3_fold_,0.836560,0.472500
3,Unet_inceptionresnetv2_fold_,0.836240,0.400000


In [4]:
test_array_for_blend = []
for model_name in tqdm_notebook(res_df['model']):
    all_by_name = list(filter(lambda x: x[5:-5] == model_name, os.listdir('prediction/')))
    pred_by_model = np.mean([np.load('prediction/' + x) for x in all_by_name], axis=0)
    test_array_for_blend.append(pred_by_model)
    
thres_array = res_df['thres'].values

blend = 0
div = len(test_array_for_blend)
for i, a in enumerate(test_array_for_blend):
    div += i
    blend += a*(len(test_array_for_blend) - i)
blend /= div

thres = 0
for i , t in enumerate(thres_array):
    thres += t*(len(thres_array) - i)
thres /= div

blend = np.int8(blend > thres)

In [5]:
create_submit(blend)

In [4]:
data = data_reader(2)

In [5]:
y_pred = np.load('prediction/val_Linknet_inceptionresnetv2_fold_2.npy')
y_valid = np.array([size_down(x) for x in data['y_valid']])
x_valid = np.array([size_down(x) for x in data['X_valid']])
y_pred_porog = np.uint8(y_pred > 0.43)

In [6]:
get_iou_vector(y_valid, y_pred_porog)

0.8540899795501022

In [7]:
import pydensecrf.densecrf as dcrf
import numpy as np
import pydensecrf.densecrf as dcrf
from skimage.io import imread, imsave
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral
from skimage.color import gray2rgb
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

def crf(original_image, mask_img):
    
    if(len(mask_img.shape)<3):
        mask_img = gray2rgb(mask_img)

    annotated_label = mask_img[:,:,0] + (mask_img[:,:,1]<<8) + (mask_img[:,:,2]<<16)
    
    colors, labels = np.unique(annotated_label, return_inverse=True)

    n_labels = 2
    
    d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels)

    U = unary_from_labels(labels, n_labels, gt_prob=0.7, zero_unsure=False)
    d.setUnaryEnergy(U)

    d.addPairwiseGaussian(sxy=(3, 3), compat=3, kernel=dcrf.DIAG_KERNEL,
                      normalization=dcrf.NORMALIZE_SYMMETRIC)
        
    Q = d.inference(10)

    MAP = np.argmax(Q, axis=0)

    return MAP.reshape((original_image.shape[0],original_image.shape[1]))

In [8]:
y_pred_new = np.array([crf(x_valid[i][:,:,0], y_pred_porog[i][:,:,0]) for i in tqdm_notebook(range(len(x_valid)))])

In [15]:
get_iou_vector(y_valid, y_pred_porog) - get_iou_vector(y_valid, y_pred_new.reshape((978, 101, 101, 1)))

0.0

In [18]:
(y_valid - y_pred_porog).mean()

0.0097500365

In [19]:
(y_valid - y_pred_new.reshape((978, 101, 101, 1))).mean()

0.010791175090296493